# Spatial

Space is defined as a network of `Location` and `Linkage`s. Generally, these can be viewed as nodes and edges. The caveat being that `Linkage`s do behave like `Location`s in some ways as they having independent activity and spatial attributes such `Resource` limitations, e.g. `Land`. 


Note: In the case of `Resource` mapping incurred due to `Transport`, currently a simplifying assumption is used where streams, e.g. `consume`, `release`, all happen at the source node. This assumption is being reviewed. The problem however is solved for across the `network`, allowing the solutions to converge



## Defining Locations

Locations are point objects in which operations such as `Process` and `Storage` can reside. These are straightforward to declare as shown in the example below where Goa is declared as the sole `Location`. Note that the single location itself serves as the `network`

In [1]:
from energia import Model, Location

m = Model()
m.Goa = Location(label='Goa, India')
m.network

Goa

Let's declare another `Location`

In [2]:
m.Pune = Location(label='Pune, India')
m.network

ntw

Given that these locations are on the same level, a network is generated with Pune and Goa as composite locations. 

In [3]:
m.network.has

(Pune, Goa)

### Nesting

Locations can be nested, take the following example

In [4]:
from energia import Model, Location

m = Model()
m.Houston = Location(label='Houston, USA')
m.CStat = Location(label='College Station, USA')
m.TX = m.Houston + m.CStat
m.TX.label = 'Texas, USA'
m.network

TX

In this case since an all encompassing `Location` exists, that is set as the network

In [5]:
m.TX.has

(CStat, Houston)

The structure can be layered

In [6]:
from energia import Model, Location

m = Model()

m.Houston = Location(label='Houston, USA')
m.CStat = Location(label='College Station, USA')
m.TX = m.Houston + m.CStat
m.TX.label = 'Texas, USA'


m.SD = Location(label='San Diego, USA')
m.LA = Location(label='Los Angeles, USA')
m.Cali = m.SD + m.LA
m.Cali.label = 'California, USA'

m.USA = m.TX + m.Cali
m.USA.label = 'United States of America'

m.Athens = Location(label='Athens, Greece')
m.Amsterdam = Location(label='Amsterdam, Netherlands')
m.EU = m.Athens + m.Amsterdam
m.EU.label = 'European Union'

m.Pune = Location(label='Pune, India')
m.Goa = Location(label='Goa, India')
m.India = m.Pune + m.Goa
m.India.label = 'India'

m.Earth = m.USA + m.EU + m.India
m.Earth.label = 'Our Home'

`Location.tree` provides a dictionary displaying the structure of the `network`

In [7]:
m.Earth.tree

{EU: {Amsterdam: {}, Athens: {}},
 USA: {Cali: {LA: {}, SD: {}}, TX: {CStat: {}, Houston: {}}},
 India: {Goa: {}, Pune: {}}}

### Constituent and Parent Locations

The parent can be seen using `Location.isin`

In [8]:
m.Goa.isin

India

`Location.has` only shows locations one order higher in disposition

In [9]:
m.USA.has

(Cali, TX)

`Location.alsohas` only shows all nested locations

In [10]:
m.USA.alsohas

(LA, SD, CStat, Houston)

## Defining Linkages

There are two ways to declare `Linkage` objects. 

### 1. Single Linkage Between Locations

If there is a single `Linkage` between two given `Locations`, `Model.Link` is more convenient. `Linkages` are always one directional. To access these linkages when providing data `location_a - location_b` or vice versa can be used

In [11]:
from energia import Model, Location, Linkage

m = Model()
m.Goa = Location(label='Goa, India')
m.Pune = Location(label='Pune, India')

m.Link(m.Goa, m.Pune)
m.linkages

[Goa-Pune]

### 2. Bi-directional Linkage Between Locations

If `bi=True` is provided as an input, a second independent `Linkage` is created

In [12]:
from energia import Model, Location, Linkage

m = Model()
m.TX = Location(label='Texas, USA')
m.CA = Location(label='California, USA')

m.Link(m.TX, m.CA, bi=True)
m.linkages

[TX-CA, CA-TX]

### 3. Multiple Linkages Between Locations

If multiple linkages exist between the same two locations, these need to be explicitly named and provided

In [13]:
from energia import Model, Location, Linkage, misc_units

m = Model(init=[misc_units])

m.USA = Location(label='USA')
m.IND = Location(label='India')

m.sea = Linkage(source=m.IND, sink=m.USA, dist=1400)
m.air = Linkage(source=m.IND, sink=m.USA, dist=8000, bi=True)
m.linkages

[sea, air, -air]

The sea route is created in one direction. Two one direction linkages are created for the air route. The negative `Linkage` can be accessed using `-Linkage` as shown

In [14]:
-m.air

-air

Negative linkages can also be declared as needed

In [15]:
m.sea_back = -m.sea
m.space.linkages  # same as Model.linkages

[sea, air, -air, sea_back]

The sibling linkage can be checked

In [16]:
m.sea.sib

sea_back

## Ascertaining Connectivity

Users can query Energia to see if two locations are connected. For a simple boolean answer, use `Location.connected()`

In [17]:
m.USA.connected(m.IND)

True

For more comprehensive information, use `Location.links()`

In [18]:
m.USA.links(m.IND)

IND is source and USA is sink in sea
IND is source and USA is sink in air
USA is source and IND is sink in -air
USA is source and IND is sink in sea_back


[sea, air, -air, sea_back]